In [1]:
import numpy as np
from PIL import Image
import os

In [2]:
def get_image(image_path):
    image = Image.open(image_path, "r")
    width, height = image.size
    pixel_values = list(image.getdata())
    if image.mode == "RGB":
        shape=(width, height, 3)
    elif image.mode == "L" or image.mode == "1":
        shape=(width, height)
    else:
        print("Unknown mode: %s" % image.mode)
        return None
    pixel_values = np.array(pixel_values).reshape((shape))
    return pixel_values

In [3]:
def zigzag_to_array(matrix):
    rows, cols = len(matrix), len(matrix[0])
    result = []
    for i in range(rows + cols - 1):
        if i % 2 == 0:
            row = 0 if i < cols else i - cols + 1
            col = i if i < cols else cols - 1
            while col >= 0 and row < rows:
                result.append(matrix[row][col])
                row += 1
                col -= 1
        else:
            col = 0 if i < rows else i - rows + 1
            row = i if i < rows else rows - 1
            while row >= 0 and col < cols:
                result.append(matrix[row][col])
                col += 1
                row -= 1
    return result


In [4]:
def zigzag_to_matrix(result, rows, cols):
    matrix = [[0 for _ in range(cols)] for _ in range(rows)]
    index = 0
    for i in range(rows + cols - 1):
        if i % 2 == 0:
            row = 0 if i < cols else i - cols + 1
            col = i if i < cols else cols - 1
            while col >= 0 and row < rows:
                matrix[row][col] = result[index]
                index += 1
                row += 1
                col -= 1
        else:
            col = 0 if i < rows else i - rows + 1
            row = i if i < rows else rows - 1
            while row >= 0 and col < cols:
                matrix[row][col] = result[index]
                index += 1
                col += 1
                row -= 1
    return matrix

In [5]:
def row_by_row_to_array(matrix):
    result = []
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            result.append(matrix[i][j])
    return result

In [6]:
def row_by_row_to_matrix(result, rows, cols):
    matrix = [[0 for _ in range(cols)] for _ in range(rows)]
    index = 0
    for i in range(rows):
        for j in range(cols):
            matrix[i][j] = result[index]
            index += 1
    return matrix

In [7]:
def col_by_col_to_array(matrix):
    result = []
    for i in range(len(matrix[0])):
        for j in range(len(matrix)):
            result.append(matrix[j][i])
    return result

In [8]:
def col_by_col_to_matrix(result, rows, cols):
    matrix = [[0 for _ in range(cols)] for _ in range(rows)]
    index = 0
    for i in range(cols):
        for j in range(rows):
            matrix[j][i] = result[index]
            index += 1
    return matrix

In [9]:
def bw_compress(arr):
    result = []
    flag = 255
    count = 0

    for i in range(len(arr)):
        if(arr[i] == flag):
            count += 1
        else:
            result.append(count)
            flag = arr[i]
            count = 1
    if(arr[i] == flag):
        result.append(count)
        flag = arr[i]
        count = 1
    return result

In [10]:
def color_compress(arr):
    result = []
    count = 1

    for i in range(1, len(arr)):
        if(arr[i] == arr[i-1]):
            count += 1
        else:
            if(count != 1):
                result.append(255)
                result.append(count)
            result.append(arr[i-1])
            count = 1
    if(arr[i] == arr[i-1]):
        result.append(255)
        result.append(count)
    result.append(arr[i])
    return result

In [11]:
def bw_decompress(compressed_data):
    color = 255
    decompressed_data = []
    for i in compressed_data:
        for j in range(i):
            decompressed_data.append(color)
            
        color = ((color + 1) % 2 ) * 255
    
    return decompressed_data

In [12]:
def color_decompress(compressed_data):
    decompressed_data = []
    i = 0
    while  i < len(compressed_data)-2:
        if(compressed_data[i] == 255):
            for _ in range(compressed_data[i+1]):
                decompressed_data.append(compressed_data[i+2])
            i += 2
        else:
            decompressed_data.append(compressed_data[i])
        i += 1
    if(i == len(compressed_data) - 2):
        decompressed_data.append(compressed_data[-2])
    decompressed_data.append(compressed_data[-1])

    return decompressed_data

In [13]:
traversal_types = {"row_by_row": (row_by_row_to_array, row_by_row_to_matrix),
                   "col_by_col": (col_by_col_to_array, col_by_col_to_matrix), 
                   "zigzag": (zigzag_to_array, zigzag_to_matrix)}
MONOCHROME_FILE_NAME = "monochrome.bmp"
GRAYSCALE_FILE_NAME = "grayscale.bmp"
RGB_FILE_NAME = "colortable.bmp"
INPUT_PATH = "assets/input/"
COMPRESSED_PATH = "assets/compressed/"
DECOMPRESSED_PATH = "assets/decompressed/"


In [14]:
def monochrome_simulation():
    monochrome_image = get_image(INPUT_PATH + MONOCHROME_FILE_NAME)

    for name, traversal in traversal_types.items():
        traversal_to_array, traversal_to_matrix = traversal
        monochrome_arr = traversal_to_array(monochrome_image)
        monochrome_compressed = bw_compress(monochrome_arr)
        with open(COMPRESSED_PATH+MONOCHROME_FILE_NAME[:-4]+"_"+name+".rle", "w") as f:
            s = " ".join(map(str, monochrome_image.shape)) + \
                "\n" + " ".join(map(str, monochrome_compressed))
            f.write(s)
        with open(COMPRESSED_PATH+MONOCHROME_FILE_NAME[:-4]+"_"+name+".rle", "r") as f:
            header = list(map(int, f.readline().strip().split(" ")))
            monochrome_compressed_from_file = list(
                map(int, f.readline().strip().split(" ")))

            monochrome_decompressed_from_file = bw_decompress(
                monochrome_compressed_from_file)
            monochrome_decompressed_matrix = traversal_to_matrix(
                monochrome_decompressed_from_file, header[0], header[1])

            image = Image.fromarray(np.array(
                monochrome_decompressed_matrix, dtype=np.uint8).reshape((header[1], header[0])))

            image.save(DECOMPRESSED_PATH +
                       MONOCHROME_FILE_NAME[:-4]+"_"+name+".bmp")
        comprasion_ratio = round(os.path.getsize(COMPRESSED_PATH+MONOCHROME_FILE_NAME[:-4]+"_"+name+".rle") / os.path.getsize(INPUT_PATH + MONOCHROME_FILE_NAME),3)
        print("Comprasion ratio of " + name + " is ", comprasion_ratio)


In [15]:
def grayscale_simulation():
    grayscale_image = get_image(INPUT_PATH + GRAYSCALE_FILE_NAME)

    for name, traversal in traversal_types.items():
        traversal_to_array, traversal_to_matrix = traversal
        grayscale_arr = traversal_to_array(grayscale_image)
        for i in range(len(grayscale_arr)):
            if (grayscale_arr[i] == 255):
                grayscale_arr[i] = 254
        grayscale_compressed = color_compress(grayscale_arr)

        with open(COMPRESSED_PATH+GRAYSCALE_FILE_NAME[:-4]+"_"+name+".rle", "w") as f:
            s = " ".join(map(str, grayscale_image.shape)) + \
                "\n" + " ".join(map(str, grayscale_compressed))
            f.write(s)
        with open(COMPRESSED_PATH+GRAYSCALE_FILE_NAME[:-4]+"_"+name+".rle", "r") as f:
            header = list(map(int, f.readline().strip().split(" ")))
            grayscale_compressed_from_file = list(
                map(int, f.readline().strip().split(" ")))

            grayscale_decompressed_from_file = color_decompress(
                grayscale_compressed_from_file)
            grayscale_decompressed_matrix = traversal_to_matrix(
                grayscale_decompressed_from_file, header[0], header[1])

            image = Image.fromarray(np.array(
                grayscale_decompressed_matrix, dtype=np.uint8).reshape((header[1], header[0])))

            image.save(DECOMPRESSED_PATH +
                       GRAYSCALE_FILE_NAME[:-4]+"_"+name+".bmp")
            
        comprasion_ratio = round(os.path.getsize(COMPRESSED_PATH+GRAYSCALE_FILE_NAME[:-4]+"_"+name+".rle") / os.path.getsize(INPUT_PATH + GRAYSCALE_FILE_NAME),3)
        print("Comprasion ratio of " + name + " is ", comprasion_ratio)


In [16]:
def rgb_simulation():
    rgb_image = get_image(INPUT_PATH + RGB_FILE_NAME)

    for name, traversal in traversal_types.items():
        traversal_to_array, traversal_to_matrix = traversal
        rgb_channels = np.transpose(rgb_image, (2, 0, 1))
        rgb_compressed_arr = []
        s = " ".join(map(str, rgb_image.shape))

        for i in range(len(rgb_channels)):
            rgb_arr = traversal_to_array(rgb_channels[i])
            for i in range(len(rgb_arr)):
                if (rgb_arr[i] == 255):
                    rgb_arr[i] = 254
            rgb_compressed_arr.append(color_compress(rgb_arr))
            s += "\n" + " ".join(map(str, rgb_compressed_arr[-1]))
        with open(COMPRESSED_PATH+RGB_FILE_NAME[:-4]+"_"+name+".rle", "w") as f:
            f.write(s)
        
        with open(COMPRESSED_PATH+RGB_FILE_NAME[:-4]+"_"+name+".rle", "r") as f:
            header = list(map(int, f.readline().strip().split(" ")))
            channels = [None] * 3
            for i in range(len(channels)):
                line = f.readline().strip().split(" ")
                rgb_compressed_from_file = list(
                    map(int, line))

                rgb_decompressed_from_file = color_decompress(
                    rgb_compressed_from_file)
                channels[i] = traversal_to_matrix(
                    rgb_decompressed_from_file, header[0], header[1])
            channels = np.asarray(channels)
            channels = np.transpose(channels, (1, 2, 0))
            image = Image.fromarray(np.array(
                 channels, dtype=np.uint8).reshape((header[1], header[0], header[2])))

            image.save(DECOMPRESSED_PATH +
                    RGB_FILE_NAME[:-4]+"_"+name+".bmp")
        
        comprasion_ratio = round(os.path.getsize(COMPRESSED_PATH+RGB_FILE_NAME[:-4]+"_"+name+".rle") / os.path.getsize(INPUT_PATH + RGB_FILE_NAME),3)
        print("Comprasion ratio of " + name + " is ", comprasion_ratio)


In [17]:
monochrome_simulation()

Comprasion ratio of row_by_row is  0.533
Comprasion ratio of col_by_col is  4.763
Comprasion ratio of zigzag is  4.719


In [18]:
grayscale_simulation()

Comprasion ratio of row_by_row is  0.935
Comprasion ratio of col_by_col is  1.932
Comprasion ratio of zigzag is  1.933


In [19]:
rgb_simulation()

Comprasion ratio of row_by_row is  3.17
Comprasion ratio of col_by_col is  3.047
Comprasion ratio of zigzag is  3.046
